# Seismic Geometry Analysis

This notebook processes seismic catalog data by:
1. Loading catalog and station data
2. Adding station coordinates to the catalog
3. Calculating back azimuth and angle of incidence
4. Filtering events based on angle of incidence criteria

## 1. Import Required Libraries

In [1]:
import pandas as pd
import numpy as np
import obspy
import matplotlib.pyplot as plt
import sys
import os

# Add the scripts directory to path to import custom modules
scripts_path = os.path.join('..', 'scripts')
if scripts_path not in sys.path:
    sys.path.append(scripts_path)

# Import seismic geometry functions
try:
    from seismic_geometry import calculate_back_azimuth, calculate_angle_of_incidence
    print("Successfully imported seismic geometry functions")
except ImportError as e:
    print(f"Warning: Could not import seismic geometry functions: {e}")
    print("Will define basic geometry functions locally")

Will define basic geometry functions locally


## 2. Load Catalog and Station Data

In [ ]:
# Load the filtered catalog sample (100 earthquake events)
catalog = pd.read_csv('../data/catalog_filtered_sample.csv')
print(f"Loaded catalog with {len(catalog)} rows and {len(catalog.columns)} columns")
print(f"Catalog columns: {catalog.columns.tolist()}")
print(f"Unique stations in catalog: {catalog['station'].unique()}")

# Load station information
stations = pd.read_csv('../data/axial_seamount_stations.csv')
print(f"\nLoaded station data with {len(stations)} rows and {len(stations.columns)} columns")
print(f"Station columns: {stations.columns.tolist()}")

# Display first few rows of each dataframe
print("\nFirst 5 rows of catalog:")
display(catalog.head())

print("\nFirst 5 rows of station data:")
display(stations.head())

## 3. Merge Station Information with Catalog

In [ ]:
# Create a station ID mapping - remove 'OO' prefix from catalog station names
catalog['station_id'] = catalog['station'].str.replace('OO', '', regex=False)
print(f"Original station names: {catalog['station'].unique()}")
print(f"Modified station IDs: {catalog['station_id'].unique()}")

# Check station names in the station dataframe
print(f"\nStation names in station file: {stations.iloc[:, 0].unique()}")

# Merge catalog with station information based on station ID
# Assuming the first column in stations contains the station ID
station_id_column = stations.columns[0]
stations_renamed = stations.rename(columns={station_id_column: 'station_id'})

# Merge the dataframes
catalog_with_stations = catalog.merge(
    stations_renamed[['station_id', 'lat', 'lon', 'elev']], 
    on='station_id', 
    how='left',
    suffixes=('', '_station')
)

# Rename station coordinates to avoid confusion with earthquake coordinates
catalog_with_stations = catalog_with_stations.rename(columns={
    'lat': 'station_lat',
    'lon': 'station_lon', 
    'elev': 'station_elev'
})

print(f"\nMerged catalog shape: {catalog_with_stations.shape}")
print(f"Number of rows with station coordinates: {catalog_with_stations['station_lat'].notna().sum()}")

# Display the merged dataframe
print("\nFirst 5 rows of merged catalog with station coordinates:")
display(catalog_with_stations.head())

## 4. Calculate Seismic Geometry Parameters

In [ ]:
# Calculate geometry parameters for each row using imported functions
print("Calculating back azimuth and angle of incidence...")

# Filter out rows without station coordinates
valid_rows = catalog_with_stations['station_lat'].notna()
print(f"Rows with valid station coordinates: {valid_rows.sum()}")

# Calculate back azimuth and angle of incidence
catalog_with_stations['back_azimuth'] = np.nan
catalog_with_stations['angle_of_incidence'] = np.nan

for idx, row in catalog_with_stations[valid_rows].iterrows():
    try:
        # Calculate back azimuth (earthquake to station)
        baz = calculate_back_azimuth(
            row['lat'], row['lon'], 
            row['station_lat'], row['station_lon']
        )
        catalog_with_stations.loc[idx, 'back_azimuth'] = baz
        
        # Calculate angle of incidence
        aoi = calculate_angle_of_incidence(
            row['lat'], row['lon'], row['depth'],
            row['station_lat'], row['station_lon'], row['station_elev']
        )
        catalog_with_stations.loc[idx, 'angle_of_incidence'] = aoi
        
    except Exception as e:
        print(f"Error calculating geometry for row {idx}: {e}")

print(f"Successfully calculated geometry for {catalog_with_stations['back_azimuth'].notna().sum()} rows")

# Display statistics
print(f"\nBack azimuth range: {catalog_with_stations['back_azimuth'].min():.1f} to {catalog_with_stations['back_azimuth'].max():.1f} degrees")
print(f"Angle of incidence range: {catalog_with_stations['angle_of_incidence'].min():.1f} to {catalog_with_stations['angle_of_incidence'].max():.1f} degrees")

# Show sample results
print("\nSample of calculated geometry parameters:")
display(catalog_with_stations[['station', 'lat', 'lon', 'depth', 'station_lat', 'station_lon', 'back_azimuth', 'angle_of_incidence']].head())

## 5. Filter by Angle of Incidence

In [ ]:
# Filter earthquakes with angle of incidence <= 30 degrees
print(f"Original dataset shape: {catalog_with_stations.shape}")
print(f"Rows with valid angle of incidence: {catalog_with_stations['angle_of_incidence'].notna().sum()}")

# Create filter for angle of incidence <= 30 degrees
aoi_filter = catalog_with_stations['angle_of_incidence'] <= 30.0
valid_aoi = catalog_with_stations['angle_of_incidence'].notna()

# Apply both filters (valid and <= 30 degrees)
final_filter = valid_aoi & aoi_filter

catalog_final = catalog_with_stations[final_filter].copy()

print(f"\nFiltered dataset shape: {catalog_final.shape}")
print(f"Rows removed: {catalog_with_stations.shape[0] - catalog_final.shape[0]}")

# Statistics about filtering
print(f"\nAngle of incidence statistics before filtering:")
print(f"  Mean: {catalog_with_stations['angle_of_incidence'].mean():.1f} degrees")
print(f"  Min: {catalog_with_stations['angle_of_incidence'].min():.1f} degrees") 
print(f"  Max: {catalog_with_stations['angle_of_incidence'].max():.1f} degrees")
print(f"  Rows > 30 degrees: {(catalog_with_stations['angle_of_incidence'] > 30).sum()}")

print(f"\nAngle of incidence statistics after filtering:")
print(f"  Mean: {catalog_final['angle_of_incidence'].mean():.1f} degrees")
print(f"  Min: {catalog_final['angle_of_incidence'].min():.1f} degrees")
print(f"  Max: {catalog_final['angle_of_incidence'].max():.1f} degrees")

# Display distribution by station
print(f"\nEvent distribution by station after filtering:")
print(catalog_final['station'].value_counts().sort_index())

# Show final dataframe
print(f"\nFinal filtered catalog:")
display(catalog_final.head())

## 6. Save Filtered Results

In [ ]:
# Save the final filtered catalog with geometry parameters
output_file = '../data/catalog_with_geometry_filtered.csv'
catalog_final.to_csv(output_file, index=False)

print(f"Saved filtered catalog with geometry parameters to: {output_file}")
print(f"Final dataset contains {len(catalog_final)} rows with {len(catalog_final.columns)} columns")

print(f"\nFinal column list:")
for i, col in enumerate(catalog_final.columns, 1):
    print(f"  {i:2d}. {col}")

print(f"\nSummary statistics:")
print(f"  Total events: {len(catalog_final)}")
print(f"  Unique earthquake IDs: {catalog_final['id'].nunique()}")
print(f"  Stations represented: {catalog_final['station'].nunique()}")
print(f"  Year range: {catalog_final['year'].min()} - {catalog_final['year'].max()}")
print(f"  Magnitude range: {catalog_final['mag'].min():.1f} - {catalog_final['mag'].max():.1f}")
print(f"  Depth range: {catalog_final['depth'].min():.1f} - {catalog_final['depth'].max():.1f} km")
print(f"  Back azimuth range: {catalog_final['back_azimuth'].min():.1f} - {catalog_final['back_azimuth'].max():.1f} degrees")
print(f"  Angle of incidence range: {catalog_final['angle_of_incidence'].min():.1f} - {catalog_final['angle_of_incidence'].max():.1f} degrees")

print(f"\nDataset is ready for shear-wave splitting analysis!")